In [1]:
from preprocessing import *
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split

2024-03-29 09:44:16.425807: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
parent_dir = './Data/In-lab/'
paths = sorted(list(os.listdir(parent_dir)))

data = []
interval_duration = 1
vec_size = int(15_000 * interval_duration)

for path in tqdm(paths):
    temp = get_data(parent_dir+path, interval_duration=interval_duration)
    dataX = temp[:, :vec_size].astype(float)    
    remaining = temp[:, vec_size:]
    # # z score normalization
    # dataX -= dataX.mean()
    # dataX /= np.std(dataX)
    # 0-1 normalization
    dataX -= dataX.min()
    dataX /= dataX.max()
    temp = np.concatenate([dataX, remaining], axis=1)
    data.append(temp)


data = np.concatenate(data, axis=0)


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


dataX, dataY = data[:,:vec_size].astype(float), data[:,-1].astype(float)
trainX, valX, trainY, valY = train_test_split(dataX, dataY)

100%|██████████| 16/16 [00:07<00:00,  2.07it/s]


In [ ]:
parent_dir = './Data/In-lab/'
paths = sorted(list(os.listdir(parent_dir)))
paths = np.array(paths)
np.random.shuffle(paths)
training_paths = paths[:-4]
validation_paths = paths[-4:]
interval_duration = 0.25
vec_size = int(15_000 * interval_duration)


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def return_dataset(paths):
    data = []

    for path in tqdm(paths):
        temp = get_data(parent_dir+path, interval_duration=interval_duration)
        dataX = temp[:, :vec_size].astype(float)    
        remaining = temp[:, vec_size:]
        # z score normalization
        dataX -= dataX.mean()
        dataX /= np.std(dataX)
        temp = np.concatenate([dataX, remaining], axis=1)
        data.append(temp)
    data = np.concatenate(data, axis=0)
    dataX = data[:, :vec_size].astype(float)
    dataY = data[:, -1].astype(float)
    dataX, dataY = unison_shuffled_copies(dataX, dataY)
    return dataX, dataY


trainX, trainY = return_dataset(training_paths)
valX, valY = return_dataset(validation_paths)

In [3]:
dataX = data[:, :vec_size].astype(float)
dataY = data[:, -1].astype(float)

In [4]:
trainX.shape

(1131, 15000)

plt.figure(figsize=(20, 6))
plt.plot(np.arange(trainX[0, :1000].size), trainX[0, :1000])
plt.show()

In [5]:
from model import get_model
import tensorflow.keras as keras


In [6]:
model = get_model(input_size=vec_size)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              )
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15000, 1)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 14998, 4)     16          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 14998, 4)    16          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv1d_1 (Conv1D)              (None, 14996, 8)     104         ['batch_normalization[0][0]']

2024-03-29 09:44:26.982752: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
from sklearn.utils import class_weight

weights = class_weight.compute_class_weight('balanced',
                                            classes=np.unique(trainY),
                                            y=trainY)
weights = {
    0: weights[0],
    1: weights[1]
}


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=1e-7, min_delta=1e-4)

history = model.fit(trainX, trainY, validation_data=(valX, valY), verbose=1, epochs=100, class_weight=weights, callbacks=[reduce_lr])

Epoch 1/100


2024-03-29 09:44:29.310418: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 122798080 exceeds 10% of free system memory.
2024-03-29 09:44:29.339738: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 122798080 exceeds 10% of free system memory.
2024-03-29 09:44:29.357995: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245563392 exceeds 10% of free system memory.
2024-03-29 09:44:29.447934: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245563392 exceeds 10% of free system memory.
2024-03-29 09:44:29.527114: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 245563392 exceeds 10% of free system memory.


36/36 [==============================] - 41s 1s/step - loss: 0.7230 - accuracy: 0.4836 - val_loss: 0.7091 - val_accuracy: 0.3148 - lr: 0.0100
Epoch 2/100
36/36 [==============================] - 39s 1s/step - loss: 0.6971 - accuracy: 0.4916 - val_loss: 0.7193 - val_accuracy: 0.3148 - lr: 0.0100
Epoch 3/100
36/36 [==============================] - 37s 1s/step - loss: 0.7013 - accuracy: 0.4978 - val_loss: 0.7541 - val_accuracy: 0.3148 - lr: 0.0100
Epoch 4/100
34/36 [===========================>..] - ETA: 1s - loss: 0.6848 - accuracy: 0.5221

In [ ]:
model.predict(trainX[:10]), trainY[:10]

In [ ]:
plt.plot(np.arange(1500), trainX[1, :1500])
plt.show()

In [ ]:
plt.plot(np.arange(200), valX[1, :200])
plt.show()